Wheat Detection EDA

https://www.kaggle.com/devvindan/wheat-detection-eda

파일

sample_submission.csv

test

train

train.csv

In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image, ImageDraw
from ast import literal_eval
import matplotlib.pyplot as plt

**데이터 로드**

In [ ]:
root_path = "../Wheat/"
train_folder = os.path.join(root_path, "train")
test_folder = os.path.join(root_path, "test")
train_csv_path = os.path.join(root_path, "train.csv")
sample_submission = os.path.join(root_path, "sample_submission.csv")

In [ ]:
df = pd.read_csv(train_csv_path)

In [ ]:
#.head() 함수를 이용하여 상위 5행을 출력
df.head()

In [ ]:
#index 0번째의 데이터 개수
df.shape[0]

**몇 가지 기초 통계**

모든 이미지의 해상도는 1024 x 1024 이다.

In [ ]:
#.unique() 개별 컬럼 내에 유일한 값을 확인, 해상도가 1024인지 확인
#width와 height가 1024이므로 결과 값 ture
df['width'].unique() == df['height'].unique() == [1024]

In [ ]:
def get_bbox_area(bbox):
    bbox = literal_eval(bbox)
    return bbox[2] * bbox[3]

In [ ]:
df['bbox_area'] = df['bbox'].apply(get_bbox_area)

In [ ]:
#bbox index 2 * index 3
df['bbox_area']

In [ ]:
#bbox index 2 * index 3의 개수 count
df['bbox_area'].value_counts()

In [ ]:
df['bbox_area'].value_counts().hist(bins=30)
plt.xlabel('number of bboxes')
plt.ylabel('area')


면적이 클수록 bbox의 개수가 작아지고, 면적이 작을수록 bbox의 개수가 커진다.

이미지마다 바운드 박스가 많이 있으며, 모든 이미지가 밀 머리와 바운드 박스를 포함하지는 않는다.

In [ ]:
unique_images = df['image_id'].unique()

In [ ]:
num_total = len(os.listdir(train_folder))
num_annotated = len(unique_images)

print(f"There are {num_annotated} annotated images and {num_total - num_annotated} images without annotations.")

주석 처리된 영상은 3373개, 주석이 없는 영상은 49개이다. (주석 처리 = 바운드 박스)

**데이터 출처**

In [ ]:
sources = df['source'].unique()
print(f"There are {len(sources)} sources of data: {sources}")

In [ ]:
df['source'].value_counts()

**각 이미지마다 몇 개의 바운드 박스가 있는지 살펴보기**

In [ ]:
plt.hist(df['image_id'].value_counts(), bins=30)
plt.xlabel('number of bboxes')
plt.ylabel('number of images')
plt.show()

train.csv에서의 바운드 박수 수 

각 이미지마다 최대 바운드 박스 수는 116개, 최소 바운드 박스 수는 1개이다.

**이미지 시각화**

In [ ]:
def show_images(images, num = 5):
    
    images_to_show = np.random.choice(images, num)

    for image_id in images_to_show:

        image_path = os.path.join(train_folder, image_id + ".jpg")
        image = Image.open(image_path)

        # get all bboxes for given image in [xmin, ymin, width, height]
        bboxes = [literal_eval(box) for box in df[df['image_id'] == image_id]['bbox']]

        # visualize them
        draw = ImageDraw.Draw(image)
        for bbox in bboxes:    
            draw.rectangle([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]], width=3)

        plt.figure(figsize = (15,15))
        plt.imshow(image)
        plt.show()

In [ ]:
show_images(unique_images)

6. 출처별 사진 보기

In [ ]:
for source in sources:
    print(f"Showing images for {source}:")
    show_images(df[df['source'] == source]['image_id'].unique(), num = 3)

**시각화를 통해 알 수 있는 내용**

* 중복된 바운드 박스가 많다.
* 사진을 수직으로 찍은 것처럼 보인다.
* 모든 식물은 다르게 회전할 수 있고, 단일한 방향은 없다.(자라는 방향이 일정하지 않다.) 이것은 flip(그림 좌우 및 상하 반전시키기)의 사용이 도움이 될 수 있다.
* 밀 머리 색은 모두 다르며 근원(source)에 좌우되는 것 같다.
* 밀 머리 자체는 관찰자와 관련된 매우 다른 각도에서 볼 수 있다.

<정리>

* 모든 이미지의 해상도는 1024 x 1024
* bbox의 면적이 클수록 bbox의 개수가 작아지고, 면적이 작을수록 bbox의 개수가 커짐
* 이미지마다 bbox가 많이 있지만, 모든 이미지가 bbox를 포함하진 않음
* bbox가 있는 이미지는 3373개, bbox가 없는 이미지는 49개임 (bbox는 곧 밀의 머리)
* train.csv에서의 각 이미지마다 최대 bbox 수는 116개, 최소 bbox 수는 1개임
* 중복되는 bbox가 많았음